<a href="https://colab.research.google.com/github/newh08/AI_Voyage/blob/main/4%E1%84%8C%E1%85%AE%E1%84%8E%E1%85%A1_%E1%84%80%E1%85%B5%E1%84%87%E1%85%A9%E1%86%AB%E1%84%80%E1%85%AA%E1%84%8C%E1%85%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace로 두 문장의 논리적 모순 분류하기

## [MYCODE] Dataset 준비

- 데이터 불러와서 확인하기.

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer

mnli_data = load_dataset("nyu-mll/glue", "mnli")
mnli_data["train"] = mnli_data["train"].shuffle(seed=42).select(range(10000))
print("----------------------Data configuration-------------------")
print(mnli_data)
train_unique_labels = mnli_data['train'].unique('label') # unique: HuggingFace Dataset 의 메서드
print("Unique labels in the train dataset:", train_unique_labels)
test_uniqe_labels = mnli_data["validation_matched"].unique('label')
print("Unique labels in the test dataset:", test_uniqe_labels)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

def preprocess_function(data):
    return tokenizer(data["premise"], data["hypothesis"], truncation=True)

mnli_tokenized = mnli_data.map(preprocess_function, batched=True)
print("----------------------Data configuration after tokenized-------------------")
print(mnli_tokenized)

# 첫 번째 샘플의 토큰화 결과 확인
print("Premise:", mnli_tokenized["train"][0]["premise"])
print("Hypothesis:", mnli_tokenized["train"][0]["hypothesis"])
print("Input IDs:", mnli_tokenized["train"][0]["input_ids"])

mnli_split = mnli_tokenized['train'].train_test_split(test_size=0.2)
mnli_train, mnli_val = mnli_split['train'], mnli_split['test']
mnli_test = mnli_tokenized['validation_matched']
print("----------------------Data length-------------------")
print(f"Train : {len(mnli_train)} , Val : {len(mnli_val)}, Test : {len(mnli_test)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

----------------------Data configuration after tokenized-------------------
DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9847
    })
})
Premise: I'll hurry over that part.
Hypothesis: "I'll be quick with that part."
Input IDs: [101, 146, 112, 1325, 10275, 1166, 1115, 1226, 119, 10

## [MYCODE] Model 구현 및 학습

사전학습된 SequenceClassification 모델 HuggingFace 에서 불러오기

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
import torch
from transformers import EarlyStoppingCallback

# 사전 학습된 distilbert 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3  # 분류 문제의 클래스 개수 설정
)

model = model.to("cuda") # 모델이 MPS 에 있으면 데이터도 mps 로 자동으로 옮겨줌.

for param in model.distilbert.parameters():
    param.requires_grad = True

training_args = TrainingArguments(
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # epoch 수
    per_device_train_batch_size=64,  # training data의 batch size
    per_device_eval_batch_size=64,  # validation data의 batch size
    logging_strategy="steps",  # steps 끝날 때마다 training loss 등을 log하라는 의미
    logging_steps=10,  # 매 10 스텝마다 로깅
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-5,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1) # 모델 예측값중 가장 높은 확률의 인덱스를 반환
    return accuracy.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mnli_train,
    eval_dataset=mnli_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-06281a2e06c0>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,1.064800,1.066399,0.428500
2,1.012400,1.046348,0.450500
3,0.998200,1.026422,0.470000
4,0.957000,1.049157,0.468500


TrainOutput(global_step=500, training_loss=1.0240736751556396, metrics={'train_runtime': 113.2875, 'train_samples_per_second': 706.168, 'train_steps_per_second': 11.034, 'total_flos': 891703637612928.0, 'train_loss': 1.0240736751556396, 'epoch': 4.0})

## [LOG] 테스트 로그
- test 정확도 0.45

In [7]:
import os
from transformers import Trainer

test_results = trainer.evaluate(eval_dataset=mnli_test)
# 평가 실행

print(test_results)


{'eval_loss': 1.036724328994751, 'eval_accuracy': 0.46897605705552725, 'eval_runtime': 7.1191, 'eval_samples_per_second': 1378.686, 'eval_steps_per_second': 21.632, 'epoch': 4.0}


## [MYCODE] SequenceClassification 을 위한 사전학습 모델 파라미터 고정해보기
- SequenceClassification 을 위한 모델의 파라미터는 고정시키면 어떤 결과가 나올지 궁금해 진행.


In [8]:
model_2 = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3
)

model_2 = model_2.to("cuda")

for param in model_2.distilbert.parameters():
    param.requires_grad = False # 파라미터 고정

trainer = Trainer(
    model=model_2,
    args=training_args,
    train_dataset=mnli_train,
    eval_dataset=mnli_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

results = trainer.evaluate(eval_dataset=mnli_test)
print(results)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-8-f97cac33a29e>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.099300,1.097976,0.347500
2,1.098800,1.097304,0.357500
3,1.099800,1.097079,0.348500
4,1.096700,1.096068,0.367500
5,1.097500,1.095712,0.363500
6,1.092100,1.095640,0.372000
7,1.094600,1.095367,0.372000
8,1.092200,1.095234,0.373500
9,1.097600,1.095112,0.374500
10,1.094000,1.095095,0.373500


{'eval_loss': 1.094952940940857, 'eval_accuracy': 0.3777890983188996, 'eval_runtime': 7.1322, 'eval_samples_per_second': 1376.159, 'eval_steps_per_second': 21.592, 'epoch': 10.0}


## [MYCODE] 파라미터 고정 해제, lr 5배 증가
- test 정확도 0.46으로 소폭 증가
- val 정확도 0.49
- 3에폭시만에 오버피팅 발생


In [9]:
model_3 = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3
)

model_3 = model_3.to("cuda")

for param in model_3.distilbert.parameters():
    param.requires_grad = True # 파라미터 고정 해제

training_args.learning_rate = 5e-5 # lr 값 증가

trainer = Trainer(
    model=model_3,
    args=training_args,
    train_dataset=mnli_train,
    eval_dataset=mnli_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

results = trainer.evaluate(eval_dataset=mnli_test)
print(results)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-4339764bdb45>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.062200,1.086702,0.414500
2,0.989800,1.034469,0.470500
3,0.942800,1.051713,0.491000


{'eval_loss': 1.036231279373169, 'eval_accuracy': 0.46979113601630157, 'eval_runtime': 7.1229, 'eval_samples_per_second': 1377.947, 'eval_steps_per_second': 21.62, 'epoch': 3.0}


## [MYCODE] 오버피팅 방지하기 위해 데이터 숫자 3배 증가
- test 정확도 0.56
- 여전히 3에폭시만에 오버피팅 발생

In [10]:
mnli_data = load_dataset("nyu-mll/glue", "mnli")
mnli_data["train"] = mnli_data["train"].shuffle(seed=42).select(range(30000))

mnli_tokenized = mnli_data.map(preprocess_function, batched=True)

mnli_split = mnli_tokenized['train'].train_test_split(test_size=0.2)
mnli_train, mnli_val = mnli_split['train'], mnli_split['test']

model_4 = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3  # 분류 문제의 클래스 개수 설정
)

model_4 = model_4.to("cuda") # 모델이 MPS 에 있으면 데이터도 mps 로 자동으로 옮겨줌.

for param in model_4.distilbert.parameters():
    param.requires_grad = True


trainer = Trainer(
    model=model_4,
    args=training_args,
    train_dataset=mnli_train,
    eval_dataset=mnli_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

results = trainer.evaluate(eval_dataset=mnli_test)
print(results)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-10-576c30700365>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.908900,0.983375,0.534833
2,0.877900,0.938380,0.567500
3,0.781400,0.968277,0.572833


{'eval_loss': 0.949315071105957, 'eval_accuracy': 0.5687213448802853, 'eval_runtime': 7.1642, 'eval_samples_per_second': 1369.997, 'eval_steps_per_second': 21.496, 'epoch': 3.0}
